In [1]:
from database import Database
from sklearn.pipeline import Pipeline
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [2]:
db = Database()

get_hotel_mapping = db.get_hotel_mapping_as_df()
get_room_mapping = db.get_room_mapping_as_df()
get_meal_mapping = db.get_meal_mapping_as_df()
get_operator_mapping = db.get_operator_mapping_as_df()
reservation = db.reservations_booking

hotel_mapping = dict(
    zip(get_hotel_mapping["external_code"], get_hotel_mapping["hotel_id"])
)
room_mapping = dict(
    zip(
        get_room_mapping.apply(lambda x: (x["external_code"], x["hotel_id"]), axis=1),
        get_room_mapping["id"],
    )
)
meal_mapping = dict(zip(get_meal_mapping["code"], get_meal_mapping["id"]))

operator_mapping = dict(
    zip(get_operator_mapping["external_code"], get_operator_mapping["operator_id"])
)

/home/jovyan/work/database.py:12: SAWarning: Did not recognize type 'geometry' of column 'geom'
  self.mapping_hotel = Table(


In [3]:
date_fr = "11/01/2022"
date_to = "10/31/2024"

In [4]:
date_fr, date_to

('11/01/2022', '10/31/2024')

In [5]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        (
            "data_encoder",
            BookingDataEncoder(
                hotel_mapping,
                room_mapping,
                meal_mapping,
                operator_mapping,
            ),
        ),
    ]
)

In [6]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 2)
    get_bookings = booking_data.get()

    if get_bookings is not None:
        df = pipeline.fit_transform(get_bookings)

    dfs.append(df)

In [9]:
bookings = pd.concat(dfs, axis=0, ignore_index=True)
bookings.to_csv("bookings.csv", index=False)

In [8]:
import pandas as pd

In [10]:
bookings.isna().sum()

ref_id                           0
res_id                           0
hotel_id                         6
operator_id                    219
operator_code                    0
bkg_ref                          0
guest_name                      29
sales_date                       0
in_date                          0
out_date                         0
days                             0
adult                            0
child                            0
purchase_price                   0
purchase_currency             1820
sales_price                      0
sales_currency                2028
purchase_price_indicator       115
sales_price_indicator          112
create_date                      0
last_modified_date               0
cancellation_date           150513
status                           0
status4                          0
status5                          0
purchase_contract_id             0
purchase_spo_id                  0
sales_contract_id                0
sales_spo_id        